In [1]:
import os
import re
from keras.preprocessing.text import Tokenizer
import numpy as np

# **Tarea 4: Modelos de lenguaje estadísticos**
## Procesamiento de lenguaje natural
### Alan García Zermeño
10 de Marzo de 2023


---

# 1.- Modelo de lenguaje y evaluación.
## 1. Preprocesamiento.
### Encontraremos primero las palabras que se repitieron más de 3 veces usando un tokenizador para fijar nuestro vocabulario.
### Después usamos la base de datos de tweets de entrenamiento para crear un corpus con palabras que estén en nuestro vocabulario, cambiando todas las que no estén por $\text{<unk>}$, y los saltos de línea por $\text{<s>}$ al principio del tweet y $\text{</s>}$ al final.


In [6]:
def get_text_from_file(path_corpus, path_truth):
  tr_txt = []
  tr_y = []

  with open(path_corpus,"r") as f_corpus, open(path_truth, "r") as f_truth:
    for twitt in f_corpus:
      tr_txt += [twitt]
    for label in f_truth:
      tr_y += [label]
  
  return tr_txt,tr_y

In [7]:
tr_txt, tr_y = get_text_from_file("/content/drive/MyDrive/Colab Notebooks/mex_data/mex_train.txt","/content/drive/MyDrive/Colab Notebooks/mex_data/mex_train_labels.txt")
test_txt, test_y = get_text_from_file("/content/drive/MyDrive/Colab Notebooks/mex_data/mex_val.txt","/content/drive/MyDrive/Colab Notebooks/mex_data/mex_val_labels.txt")

In [8]:
import nltk
from nltk.tokenize import TweetTokenizer
tokenizer = TweetTokenizer()

In [9]:
corpus_palabras = []
for doc in tr_txt:
  corpus_palabras += tokenizer.tokenize(doc)

#corpus_palabras = [t for t in corpus_palabras if t not in stopwords]
fdist = nltk.FreqDist(corpus_palabras)
len(fdist)

13581

In [10]:
#Defino mi vocabulario
V = [key for key in fdist if key[0] not in ["#","/"] and fdist[key] > 2]
V.append('</s>')
V.append('<s>')
V.append('<unk>')
len(V)

3102

In [11]:
def preproces(txt):
  corpus = []
  for doc in txt:
    corpus.append("<s>")
    for word in tokenizer.tokenize(doc):
      if word not in V:
        corpus.append("<unk>")
      else:
        corpus.append(word)
    corpus.append("</s>")
  return(corpus)

In [12]:
tr_txt_val = tr_txt[len(tr_txt)-len(test_txt):] #Validation data
tr_txt = tr_txt[:len(tr_txt)-len(test_txt)] #Training data

corp_prepro = preproces(tr_txt)
prepro_val = preproces(tr_txt_val)
prepro_test = preproces(test_txt)
#txt_corp = nltk.text.Text(corp_prepro)
print(corp_prepro[:30])

['<s>', 'lo', 'peor', 'de', 'todo', 'es', 'que', 'no', 'me', 'dan', 'por', 'un', 'tiempo', 'y', 'luego', 'vuelven', 'estoy', 'hasta', 'la', 'verga', 'de', '<unk>', '</s>', '<s>', 'a', 'la', 'vga', 'no', 'seas', 'mamón']


## 2. Modelo de N-gramas

In [13]:
def c_base(corp,words):
  return(0)

In [14]:
fdist_cor = nltk.FreqDist(corp_prepro)   #Frecuencia de unigramas

def c_uni(corp,word):
  return(fdist_cor[word[0]])

In [15]:
tt = nltk.bigrams(corp_prepro)
fdist_cor_bi = nltk.FreqDist(tt)  #frecuencia de bigramas

def c_big(corp,words):
  return(fdist_cor_bi[(words[0],words[1])])

In [16]:
find = nltk.collocations.TrigramCollocationFinder.from_words(corp_prepro)
trigrams = find.ngram_fd  #Frecuencia de trigramas

In [17]:
def c_tri(corp,words):
  coun = trigrams[(words[0],words[1],words[2])]
  return(coun)

In [18]:
def P_markov(corp,phrase,ngram,n):
  g = [c_base,c_uni,c_big,c_tri]
  ph = phrase#.split()
  p = 0
  kk = 0.01
  m = n*kk

  if len(ph) < ngram:
    print("Error")
    return 0

  #Check for missing words in vocabulary and replace
  for i in range(len(ph)):
    if ph[i] not in V:
      ph[i] = "<unk>"
  
  #Compute ngram markov chain probability
  for i in range(len(ph)-ngram+1):
      #Estas condiciones son para casos que no vale la pena contar
      if ngram > 1 and ph[i:i+ngram][0] == "</s>":
        pass
      if ngram == 3 and ph[i:i+ngram][1] in ["<s>","</s>"]:
        pass
      else:    
        if ngram > 1:
          pn = g[ngram](corp,ph[i:i+ngram])
          p += np.log2((pn+kk)/(g[ngram-1](corp,ph[i:i+ngram-1]) + m))
        else:
          p += np.log2((c_uni(corp,[ph[i]])+kk)/(n+m))
          #print(np.power(2,np.log2((c_uni(corp,[ph[i]])+kk)/m)),m,n)
    #print("c(",ph[i:i+ngram],") =",pn)

  return(p) 

## Se optó por un suavizado de Laplace para lidiar con los conteos nulos y para el uso de la función P_markov enviamos en forma de arreglo la cadena a la cual queremos calcular su probabilidad, y con el último argumento definimos si queremos usar unigramas (1), bigramas(2) o trigramas (3). La función devuelve el logaritmo base 2 de la probabilidad calculada, por lo que calculamos $2^{\log_2 p}$, para encontrar la probabilidad de 0 a 1.

In [352]:
c_uni(prepro_val, ["psicólogo"])

0

In [353]:
c_big(prepro_val,["viva","méxico"])

7

In [19]:
c_tri(prepro_val,["viva","méxico","</s>"])

1

In [20]:
ex1 = P_markov(corp_prepro, ["viva" ,"méxico" ,"</s>"],1,len(corp_prepro))
ex2 = P_markov(corp_prepro, ["viva" ,"méxico" ,"</s>"],2,len(corp_prepro))
ex3 = P_markov(corp_prepro, ["viva" ,"méxico" ,"</s>"],3,len(corp_prepro))
print("P_uni = ",np.power(2,ex1),"\nP_big = ",np.power(2,ex2) ,"\nP_trig = ",np.power(2,ex3))

P_uni =  4.3015661126774835e-09 
P_big =  2.59862107786755e-05 
P_trig =  0.0010035372203012593


In [21]:
ex1 = P_markov(corp_prepro, ["mi" ,"psicólogo","dice"],1,len(corp_prepro))
ex2 = P_markov(corp_prepro, ["mi" ,"psicólogo","dice"],2,len(corp_prepro))
ex3 = P_markov(corp_prepro, ["mi" ,"psicólogo","dice"],3,len(corp_prepro))
print("P_uni = ",np.power(2,ex1),"\nP_big = ",np.power(2,ex2) ,"\nP_trig = ",np.power(2,ex3))

P_uni =  4.6245754568951226e-07 
P_big =  5.277325231034305e-05 
P_trig =  0.0009144906015718378


## 3. Análisis de Perplejidad con un modelo $\hat{P}$ interpolado de la forma:
## $\hat{P}(w_n|w_{n-2}w_{n-1}) = \lambda_1 P(w_n|w_{n-2}w_{n-1}) + \lambda_2 P(w_n|w_{n-1}) + \lambda_3 P(w_n)$
## $Prp = 2^{-\frac{1}{N}∑_i^n \log_2 ~p_i}$

In [22]:
def Prpx(N,p):
  return(np.power(2,-(1/N)*p))

In [24]:
#Sobre datos de validación
N = len(prepro_val)
ei = np.array([P_markov(corp_prepro, prepro_val,1,len(V)),
               P_markov(corp_prepro, prepro_val,2,len(V)),
               P_markov(corp_prepro, prepro_val,3,len(V))])

#Pi = np.power(2,ei[0])
lamb = np.array([[1/3,1/3,1/3],[0.4,0.4,0.2],[0.2,0.4,0.4],
                 [0.5,0.4,0.1],[0.1,0.4,0.5]])

P_interp = [np.sum(lam*ei) for lam in lamb]
print("Perplejidad de interpolación con distintos valores de lambda:\n")
print(Prpx(N,ei))
for i,p in enumerate(P_interp):
  print("Prpx(lam = ",lamb[i],") = ", Prpx(N,p))

Perplejidad de interpolación con distintos valores de lambda:

[  7.33534972 142.14086186 341.86752198]
Prpx(lam =  [0.33333333 0.33333333 0.33333333] ) =  70.90320565347504
Prpx(lam =  [0.4 0.4 0.2] ) =  51.7639580016772
Prpx(lam =  [0.2 0.4 0.4] ) =  111.61302535542941
Prpx(lam =  [0.5 0.4 0.1] ) =  35.25200103261191
Prpx(lam =  [0.1 0.4 0.5] ) =  163.89231214403225


In [25]:
#Sobre datos de prueba
N = len(prepro_test)
ei = np.array([P_markov(corp_prepro, prepro_test,1,len(V)),
               P_markov(corp_prepro, prepro_test,2,len(V)),
               P_markov(corp_prepro, prepro_test,3,len(V))])

#Pi = np.power(2,ei[0])
lamb = np.array([[1/3,1/3,1/3],[0.4,0.4,0.2],[0.2,0.4,0.4],
                 [0.5,0.4,0.1],[0.1,0.4,0.5]])

P_interp = [np.sum(lam*ei) for lam in lamb]
print(Prpx(N,ei))
print("Perplejidad de interpolación con distintos valores de lambda:\n")
for i,p in enumerate(P_interp):
  print("Prpx(lam = ",lamb[i],") = ", Prpx(N,p))

[  6.09775253 120.68490018 293.85119705]
Perplejidad de interpolación con distintos valores de lambda:

Prpx(lam =  [0.33333333 0.33333333 0.33333333] ) =  60.02286642828897
Prpx(lam =  [0.4 0.4 0.2] ) =  43.687237647332196
Prpx(lam =  [0.2 0.4 0.4] ) =  94.83008043029005
Prpx(lam =  [0.5 0.4 0.1] ) =  29.65232954908213
Prpx(lam =  [0.1 0.4 0.5] ) =  139.71463027942588


### Sobre los datos de validación y sobre los de test el modelo interpolado de $\lambda = [0.5,0.4,0.1]$ funcionó mejor, así como en los datos de prueba. En ambos casos vemos que el modelo de unigramas fue el que más probabilidad calculó.
### Se esperaba que la perplejidad de los trigramas fuera menor, sin embargo pudo haber afectado el uso del suavizado de Laplace. Previamente se hicieron pruebas contando combinaciones de tokens como c($</s>$,palabra), pero la perplejidad de los trigramas y bigramas bajó bastante cuando se dejaron de contar.

# 3. Generación de texto

## 2. Usando valores fijos de $\lambda$, generamos tweets de máximo 50 palabras, sampleando sobre un arreglo de las posibles palabras a generar según los conteos de bigramas y seleccionando la siguiente palabra con base en las probabilidades interpoladas de cada una de ellas.
## Para ello, usamos valores de $\lambda$ que le quiten peso a los unigramas. Por ello (y por falta de tiempo), no se programó la función de EM, ya que, por las perplejidades calculadas previamente, éste algoritmo iba a darle todo el peso a los unigramas.
## Imprimimos 5 tweets generados con 2 palabras de interés cada uno para empezar a generar el tweet.

In [26]:
lamb1 = 0.1
lamb2 = 0.4
lamb3 = 0.5

def tuitear(corp,iter,tw):
  no = ["</s>","<s>","<unk>","|","'",'"',"¡","¿","."]
  p = {}

  for jj in range(iter):
    p.clear()
    if jj == 30:
      no.pop(0)

    #Creamos una lista de palabras candidatas
    for word in V:
      if word not in no:      
        big = c_big(corp,[tw[-1],word])
        if big > 1:
          p[word] = big
    #print(len(p),tw[-1])

    for word in p:
      uni = P_markov(corp,[word],1,len(corp))
      big = P_markov(corp,[tw[-1],word],2,len(corp))
      tri = P_markov(corp,tw[-2:]+[word],3,len(corp))
      pg = lamb1*np.power(2,uni) + lamb2*np.power(2,big) + lamb3*np.power(2,tri)
      p[word] = pg

    sum = np.sum(list(p.values()))
    t = 0
    r = np.random.uniform(0,1)
    #print(len(p),tw[-1])

    for w in p:
      p[w] = p[w]/sum + t
      if r < p[w] and r > t:
        if w == "</s>":
          return " ".join(tw)
        tw.append(w)
        break
      t = p[w]
  return(" ".join(tw))

In [38]:
print("1.- ",tuitear(corp_prepro,50,["no","creo"]))
print("2.- ",tuitear(corp_prepro,50,["yo","soy"]))
print("3.- ",tuitear(corp_prepro,50,["estoy","bien"]))
print("4.- ",tuitear(corp_prepro,50,["te","voy"]))
print("5.- ",tuitear(corp_prepro,50,["me","gusta"]))

1.-  no creo que se nos va a putas rotaciones
2.-  yo soy el día para qué putas ratas que me digan esos putos corruptos de la madre porque putas ! el no mamen
3.-  estoy bien la lengua en 8 putos narizones de esa madre yo siempre está su puta madre el hijo de puta madre ... ni a su novio el mundial a la vida y todos los de “ es que tengo en la madre ! ! por qué putas van a nadie más
4.-  te voy a la escuela de un hombre se quejan
5.-  me gusta de honduras y que me cagan las mujeres : v ) ... para mandarte a los putos argentinos


## 3. Datos de la mañanera de AMLO
### Se tomaron ahora los datos de la mañanera de AMLO, debido al tamaño del corpus, todo el preprocesamiento resultó muy tardado. Pero se usó el mismo método para definir el vocabulario que en el corpus de los tweets (Palabras repetidas más de n ocasiones).


In [165]:
import glob

amloConf = ""
for f_pagina in glob.glob("/content/drive/MyDrive/Colab Notebooks/estenograficas_limpias_por_fecha/*"):
    amloConf += (open(f_pagina,"r",encoding = "utf-8").read())[800:]

amloConfTok = amloConf.split()
len(amloConfTok)

11001741

In [170]:
amloConfTok = tokenizer.tokenize(amloConf)
fdist_AMLO = nltk.FreqDist(amloConfTok)
len(fdist_AMLO)

94324

In [173]:
V_AMLO = [key for key in fdist_AMLO if key[0] not in ["#","/"] and fdist_AMLO[key] > 4]
#V_AMLO.append('</s>')
#V_AMLO.append('<s>')
V_AMLO.append('<unk>')
len(V_AMLO)

37737

In [176]:
def preproces_txt(txt):
  corpus = []
    #corpus.append("<s>")
  for word in txt:
    if word not in V_AMLO:
      corpus.append("<unk>")
    else:
      corpus.append(word)
    #corpus.append("</s>")
  return(corpus)

In [177]:
AMLO_prepro = preproces_txt(amloConfTok)
AMLO_prepro[:30]

['<unk>',
 '.',
 '18',
 'Versión',
 'estenográfica',
 'de',
 'la',
 'firma',
 'del',
 'Decreto',
 'de',
 'Estímulos',
 '<unk>',
 'de',
 'la',
 'Región',
 '<unk>',
 'Norte',
 'VE',
 '<unk>',
 'Versión',
 'estenográfica',
 'San',
 'Pedro',
 'Garza',
 'García',
 ',',
 'Nuevo',
 'León',
 ',']

In [179]:
fdist_cor = nltk.FreqDist(AMLO_prepro)   #Frecuencia de unigramas
tt = nltk.bigrams(AMLO_prepro)
fdist_cor_bi_AM = nltk.FreqDist(tt)  #frecuencia de bigramas

find = nltk.collocations.TrigramCollocationFinder.from_words(AMLO_prepro)
trigrams_AM = find.ngram_fd  #Frecuencia de trigramas

In [180]:
def c_uni(corp,word):
  return(fdist_cor[word[0]])

def c_big(corp,words):
  return(fdist_cor_bi_AM[(words[0],words[1])])

def c_tri(corp,words):
  coun = trigrams_AM[(words[0],words[1],words[2])]
  return(coun)

### Primero usamos la misma estrategia de generación de tweets para generar una conferencia de 300 palabras iniciada por "Buenos días". El resultado no es suficientemente satisfactorio, podemos encontrar pequeñas frases conexas y con sentido pero en general no podemos decir que es un buen modelo.

In [344]:
lamb1 = 0.1
lamb2 = 0.4
lamb3 = 0.5

def conference(corp,iter):
  tw = ["buenos","días"]
  no = ["</s>","<s>","<unk>","|","'",'"',"¡","¿"]
  p = {}

  for jj in range(iter):
    p.clear()

    #Creamos una lista de palabras candidatas
    for word in V_AMLO:
      if word not in no:      
        big = c_big(AMLO_prepro,[tw[-1],word])
        pg = np.power(2,big)
        if pg > 3:
          p[word] = pg
    
    #Le interpolamos una probabilidad a cada una
    for word in p:
      uni = P_markov(corp,[word],1,len(AMLO_prepro))
      big = P_markov(corp,[tw[-1],word],2,len(AMLO_prepro))
      tri = P_markov(corp,tw[-2:]+[word],3,len(AMLO_prepro))
      p[word] = lamb1*np.power(2,uni) + lamb2*np.power(2,big) + lamb3*np.power(2,tri)

    #Normalizamos y sampleamos una de las palabras
    sum = np.sum(list(p.values()))
    t = 0
    r = np.random.uniform(0,1)
    for w in p:
      p[w] = p[w]/sum + t
      if r < p[w] and r > t:
        tw.append(w)
        break
      t = p[w]

  return(" ".join(tw))

In [345]:
conference(AMLO_prepro,300)

'buenos días vimos las regalías , desató la conquista a Múzquiz . Empresas que subirle , colosal , valía el carbón de Evo a especializarse en eficiencia en grado . BLAS : Sé de vanidad , igualdad … VOZ : Cassez y clasismo . Tomen nota Jesús si regresaran a detallar . INTELOCUTOR : liberales en francos , predominó por influencias de Cristo , protestaron , hidráulica y cometen actos heroicos de previsiones de cierres y dinamizar la Panamericana y CorpoGas , válido y casinos . Hago notar la Farmacia Benavides y cimentación de instalarse . Chichén Itzá , facilidades , empiecen , conseguir alimentos es Elektra y numerario , protegido a Conapesca para secretario se polarizó , aduanas no hará la deducción de directora . Pudiera pensarse que asistir en tratos y levantaban la eliminación , asesinatos en Relaciones que definirnos , 508 , Maribel Villegas y distribuyendo . EDGAR JESÚS CRUZ ESCANDÓN CADENAS , poniéndose de 114 en recorrido de excavaciones . EDUARDO REDONDO ARÁMBURO , procuremos qu

## 4. Calculamos ahora la probabilidad para nuestros dos modelos, 2 frases distintas: 
## *sino gano me voy a la chingada*

Modelo de AMLO

In [309]:
ex1 = P_markov(AMLO_prepro, ["sino" ,"gano" ,"me","voy","a","la","chingada"],1,len(AMLO_prepro))
ex2 = P_markov(AMLO_prepro, ["sino" ,"gano" ,"me","voy","a","la","chingada"],2,len(AMLO_prepro))
ex3 = P_markov(AMLO_prepro, ["sino" ,"gano" ,"me","voy","a","la","chingada"],3,len(AMLO_prepro))
print("P_uni = ",np.power(2,ex1),"\nP_big = ",np.power(2,ex2) ,"\nP_trig = ",np.power(2,ex3))

P_uni =  1.1854752675745273e-24 
P_big =  1.1834976981245866e-20 
P_trig =  6.761651376528221e-28


Modelo de tweets

In [39]:
ex1 = P_markov(corp_prepro, ["sino" ,"gano" ,"me","voy","a","la","chingada"],1,len(corp_prepro))
ex2 = P_markov(corp_prepro, ["sino" ,"gano" ,"me","voy","a","la","chingada"],2,len(corp_prepro))
ex3 = P_markov(corp_prepro, ["sino" ,"gano" ,"me","voy","a","la","chingada"],3,len(corp_prepro))
print("P_uni = ",np.power(2,ex1),"\nP_big = ",np.power(2,ex2) ,"\nP_trig = ",np.power(2,ex3))

P_uni =  4.764022901954099e-17 
P_big =  1.3218064657063327e-13 
P_trig =  9.228839269547681e-17


## *ya se va a acabar la corrupción*

Modelo de AMLO

In [292]:
ex1 = P_markov(AMLO_prepro, ["ya" ,"se" ,"va","a","acabar","la","corrupción"],1,len(AMLO_prepro))
ex2 = P_markov(AMLO_prepro, ["ya" ,"se" ,"va","a","acabar","la","corrupción"],2,len(AMLO_prepro))
ex3 = P_markov(AMLO_prepro, ["ya" ,"se" ,"va","a","acabar","la","corrupción"],3,len(AMLO_prepro))
print("P_uni = ",np.power(2,ex1),"\nP_big = ",np.power(2,ex2) ,"\nP_trig = ",np.power(2,ex3))

P_uni =  2.9866736314444386e-18 
P_big =  9.125692563030109e-14 
P_trig =  5.068400071230098e-16


Modelo de tweets

In [40]:
ex1 = P_markov(corp_prepro, ["ya" ,"se" ,"va","a","acabar","la","corrupción"],1,len(corp_prepro))
ex2 = P_markov(corp_prepro, ["ya" ,"se" ,"va","a","acabar","la","corrupción"],2,len(corp_prepro))
ex3 = P_markov(corp_prepro, ["ya" ,"se" ,"va","a","acabar","la","corrupción"],3,len(corp_prepro))
print("P_uni = ",np.power(2,ex1),"\nP_big = ",np.power(2,ex2) ,"\nP_trig = ",np.power(2,ex3))

P_uni =  1.171122011735223e-19 
P_big =  3.055792160215833e-17 
P_trig =  2.6902840572272688e-20


## 5. Por último, calculamos la probabilidad de todas las permutaciones posibles de ambas frases y mostramos el top 5 de permutaciones más probables.
## *ya se va a acabar la corrupción*

In [41]:
import itertools
def permutations(corp,chain,n,k):
  d = {}
  perm = list(itertools.permutations(chain))
  for prm in perm:
    ex1 = np.power(2,P_markov(corp,list(prm),1,n))
    ex2 = np.power(2,P_markov(corp,list(prm),2,n))
    ex3 = np.power(2,P_markov(corp,list(prm),3,n))
    d[prm] = lamb1*ex1 + lamb2*ex2 + lamb3*ex3
  d = dict(sorted(d.items(), key=lambda item: item[1],reverse=True))
  return([(list(d)[i],list(d.values())[i]) for i in range(k)])

Modelo de AMLO

In [301]:
g = permutations(AMLO_prepro,["ya" ,"se" ,"va","a","acabar","la","corrupción"],len(AMLO_prepro),5)
for i in g: print(i[0],i[1])

('ya', 'se', 'va', 'a', 'acabar', 'la', 'corrupción') 2.7631095027378115e-14
('ya', 'la', 'corrupción', 'se', 'va', 'a', 'acabar') 1.773150099266756e-14
('la', 'corrupción', 'ya', 'se', 'va', 'a', 'acabar') 6.909040387394665e-15
('acabar', 'la', 'corrupción', 'ya', 'se', 'va', 'a') 5.877799493567909e-15
('acabar', 'ya', 'se', 'va', 'a', 'la', 'corrupción') 2.4885907121919703e-15


Modelo de Tweets

In [42]:
g = permutations(corp_prepro,["ya" ,"se" ,"va","a","acabar","la","corrupción"],len(corp_prepro),5)
for i in g: print(i[0],i[1])

('ya', 'se', 'va', 'a', 'la', 'corrupción', 'acabar') 4.200989711920996e-15
('acabar', 'ya', 'se', 'va', 'a', 'la', 'corrupción') 4.1884858738444555e-15
('ya', 'a', 'la', 'corrupción', 'se', 'va', 'acabar') 2.8547078683751045e-15
('a', 'la', 'corrupción', 'ya', 'se', 'va', 'acabar') 1.8058786830128012e-16
('ya', 'se', 'va', 'acabar', 'a', 'la', 'corrupción') 1.8005015007962087e-16


## *sino gano me voy a la chingada*

Modelo de AMLO

In [302]:
g = permutations(AMLO_prepro,["sino" ,"gano" ,"me","voy","a","la","chingada"],len(AMLO_prepro),5)
for i in g: print(i[0],i[1])

('chingada', 'sino', 'me', 'voy', 'a', 'la', 'gano') 1.4028522006077076e-19
('sino', 'me', 'voy', 'a', 'la', 'gano', 'chingada') 7.955677340304373e-20
('chingada', 'me', 'voy', 'a', 'la', 'gano', 'sino') 1.8496458922101176e-20
('chingada', 'sino', 'me', 'voy', 'a', 'gano', 'la') 1.825893067711217e-20
('me', 'voy', 'a', 'la', 'gano', 'sino', 'chingada') 1.775165275272428e-20


Modelo de Tweets

In [43]:
g = permutations(corp_prepro,["sino" ,"gano" ,"me","voy","a","la","chingada"],len(corp_prepro),5)
for i in g: print(i[0],i[1])

('me', 'voy', 'a', 'gano', 'sino', 'la', 'chingada') 2.954160836354483e-13
('me', 'voy', 'a', 'la', 'chingada', 'gano', 'sino') 1.7468882169312703e-13
('sino', 'la', 'chingada', 'a', 'gano', 'me', 'voy') 1.6200583269956476e-13
('sino', 'la', 'chingada', 'gano', 'me', 'voy', 'a') 9.732983789052444e-14
('sino', 'me', 'voy', 'a', 'gano', 'la', 'chingada') 9.300765819761783e-14


## Al observar las probabilidades de las permutaciones, comprobamos que los modelos no son suficientemente buenos para contemplar frases largas, las probabilidades de los trigramas y bigramas más comunes hacen que las permutaciones menos lógicas tengan mayor probabilidad.